In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

In [3]:
folder_path = './data/names'

file_list = glob.glob(os.path.join(folder_path, '*.txt'))

print(len(file_list))

names_df_list = []

for file in file_list:
    data = pd.read_csv(file, delimiter=',', header=None)
    year = int(os.path.basename(file).split('.')[0][-4:])
    data = pd.concat([data, pd.Series([year] * len(data))], axis=1)
    names_df_list.append(data)

143


In [4]:
names_df = pd.concat(names_df_list, ignore_index=True)

column_names = ['Name', 'Sex', 'Quanity', 'Year']

names_df.columns = column_names

print(names_df)

             Name Sex  Quanity  Year
0        Jennifer   F    56323  1978
1         Melissa   F    28328  1978
2         Jessica   F    26103  1978
3             Amy   F    23212  1978
4         Heather   F    22270  1978
...           ...  ..      ...   ...
2085153      Zeph   M        5  2000
2085154     Zeven   M        5  2000
2085155     Ziggy   M        5  2000
2085156        Zo   M        5  2000
2085157     Zyier   M        5  2000

[2085158 rows x 4 columns]


In [5]:
num_uniq_names = len(names_df['Name'].unique())
num_uniq_names_female = len(names_df.loc[names_df['Sex'] == 'F']['Name'].unique())
num_uniq_names_male = len(names_df.loc[names_df['Sex'] == 'M']['Name'].unique())

print("Number of unique names across data:", num_uniq_names)
print("Number of unique Female names across data:", num_uniq_names_female)
print("Number of unique Male names across data:", num_uniq_names_male)

Number of unique names across data: 102449
Number of unique Female names across data: 70227
Number of unique Male names across data: 43655


In [60]:
names_df_pivot = names_df.pivot_table(index=['Year', 'Sex', 'Name'], values='Quanity')

names_male_freq = names_df_pivot.loc[(slice(None), 'M'), :].apply(lambda x: x.values/x.sum())
names_female_freq = names_df_pivot.loc[(slice(None), 'F'), :].apply(lambda x: x.values/x.sum())

names_df_pivot['frequency_male'] = names_male_freq
names_df_pivot['frequency_female'] = names_female_freq

names_df_pivot

Quanity  frequency_male  frequency_female
Year Sex Name                                              
1880 F   Abbie       71.0             NaN      3.929584e-07
         Abby         6.0             NaN      3.320775e-08
         Abigail     12.0             NaN      6.641550e-08
         Ada        652.0             NaN      3.608576e-06
         Adah        24.0             NaN      1.328310e-07
...                   ...             ...               ...
2022 M   Zyren        8.0    4.333331e-08               NaN
         Zyrie       11.0    5.958330e-08               NaN
         Zyrin        6.0    3.249998e-08               NaN
         Zyron       12.0    6.499997e-08               NaN
         Zyrus        8.0    4.333331e-08               NaN

[2085158 rows x 3 columns]